In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset'

/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset


In [3]:
!mkdir -p raw/quora raw/parabank neg indices

In [4]:
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 64.4 MB/s 
     |████████████████████████████████| 120 kB 49.4 MB/s 
     |████████████████████████████████| 212 kB 52.3 MB/s 
     |████████████████████████████████| 127 kB 50.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
import re
import datasets
import os
import pandas as pd
import numpy as np

In [6]:
def para(row, anchor_col="sentence1", para_col="sentence2", label_col="label", path="paws", name="small_diverse", split="train"):
  out_row = {
             "sentence1": row[anchor_col],
             "sentence2": row[para_col],
             "label": row[label_col],
             "path": path,
             "name": name,
             "split": split}
  return out_row

dataset_definitions = [
    {"path": "paws", "name": "labeled_final"},
    {"path": "paws", "name": "labeled_swap"},
    {"path": "paws", "name": "unlabeled_final"}]

for definition in dataset_definitions:
    # load max 50000 samples, only affects "unlabeled_final", all others are smaller
    dataset = datasets.load_dataset(path=definition["path"], name=definition["name"]).filter(lambda example: example["id"] <= 50000)
    for split in ["train", "validation", "test"]:
        out = []
        if split not in dataset:
            continue
        dataset_part = pd.DataFrame(dataset[split])
        #dataset_post = dataset_part.map(lambda x: para(model, tokenizer, x), num_proc=1)
        for idx, row in dataset_part.iterrows():
            out.append(para(row, split=split, path=definition["path"], name=definition["name"]))
        df = pd.DataFrame(out)
        #df = df[df["original_label"] == 1]
        df = df.reset_index()   
        df.to_csv(f'raw/out_{definition["path"]}_{definition["name"]}_{split}.csv', index=False)



Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dataset paws downloaded and prepared to /root/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset paws downloaded and prepared to /root/.cache/huggingface/datasets/paws/unlabeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/646 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]